.. gps-receivers:

## GPS Receivers

Most GPS receivers have data logging capabilities that you can use with Pipecat to view navigational information.  Some receivers connect to your computer via a serial port or a serial-over-USB cable that acts like a traditional serial port.  Others can push data to a network socket.  For this demonstration, we will receive GPS data sent from an iPhone to a UDP socket:

In [1]:
# nbconvert: hide
from __future__ import absolute_import, division, print_function

import pipecat.test
socket = pipecat.test.mock_module("socket")

path = "../data/gps"
client = "172.10.0.20"

socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, stop=6)

In [2]:
import pipecat.record
import pipecat.udp
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
for record in pipe:
    pipecat.record.dump(record)

client: 172.10.0.20
string: $GPTXT,01,01,07,Pipecat*12


client: 172.10.0.20
string: $GPGGA,164100,3511.33136,N,10643.48435,W,1,8,0.9,1654.0,M,46.9,M,0,2*50


client: 172.10.0.20
string: $GPRMC,164100,A,3511.33136,N,10643.48435,W,0.00,0.00,311216,003.1,W*7C


client: 172.10.0.20
string: $GPGLL,3511.33136,N,10643.48435,W,164100,A*36


client: 172.10.0.20
string: $HCHDG,129.5,,,8.7,E*29


client: 172.10.0.20
string: $PASHR,164100190,138.24,T,+32.56,+48.49,+00.00,3.141,3.141,35.000,1,0*17




Here, we used :func:`pipecat.udp.receive` to open a UDP socket listening on port 7777 on all available network interfaces ("0.0.0.0") and convert the received messages into Pipecat :ref:`records`, which we dump to the console.  Note that each record includes the IP address of the client (the phone in this case), along with a string containing the raw data of the message.  In this case the raw data is in NMEA format, a widely-used standard for exchanging navigational data.  To decode the contents of each message, we add the appropriate Pipecat device to the end of the pipe:

In [3]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, stop=6)

In [4]:
import pipecat.device.gps
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    pipecat.record.dump(record)

id: GPTXT
text: Pipecat

altitude: 1654.0 meter
dop: 0.9
geoid-height: 46.9 meter
id: GPGGA
latitude: 35.188856 degree
longitude: -106.724739167 degree
quality: 1
satellites: 8
time: 164100

active: True
date: 311216
id: GPRMC
latitude: 35.188856 degree
longitude: -106.724739167 degree
speed: 0.0 knot
time: 164100
track: 0.0 degree
variation: -3.1 degree

active: True
id: GPGLL
latitude: 35.188856 degree
longitude: -106.724739167 degree
time: 164100

heading: 129.5 degree
id: HCHDG
variation: 8.7 degree

heading: 138.24 degree
heading-accuracy: 35.0 degree
heave: 0.0 meter
id: PASHR
pitch: 48.49 degree
pitch-accuracy: 3.141 degree
roll: 32.56 degree
roll-accuracy: 3.141 degree
time: 164100190



As you can see, :func:`pipecat.device.gps.nmea` has converted the raw NMEA messages into records containing human-readable navigational fields with appropriate physical units.  Note that unlike the :ref:`battery-chargers` example, not every record produced by the GPS receiver has the same fields.  The NMEA standard includes many different *types* of messages, and most GPS receivers will produce more than one type.  This will increase the complexity of our code - for example, we might have to test for the presence of a field before extracting it from a record:

In [5]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=110)

In [6]:
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    if "latitude" in record:
        print("Latitude:", record["latitude"], "Longitude:", record["longitude"])

Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1949926667 degree Longitude: -106.7111135 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree
Latitude: 35.1952843333 degree Longitude: -106.710192667 degree


Alternatively, we might use the record `id` field to key our code off a specific type of NMEA message:

In [7]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=120)

In [8]:
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    if record["id"] == "PASHR":
        print("Pitch:", record["pitch"])

Pitch: 66.82 degree
Pitch: 67.3 degree
Pitch: 66.8 degree
Pitch: 66.18 degree


Another alternative would be to add a filter to our pipe so we only receive records that match some criteria:

In [9]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=120)

In [10]:
import pipecat.filter
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
pipe = pipecat.filter.keep(pipe, key="id", value="GPGLL")
for record in pipe:
    pipecat.record.dump(record)

active: True
id: GPGLL
latitude: 35.1949926667 degree
longitude: -106.7111135 degree
time: 164252

active: True
id: GPGLL
latitude: 35.1952843333 degree
longitude: -106.710192667 degree
time: 164257

active: True
id: GPGLL
latitude: 35.1956116667 degree
longitude: -106.709064 degree
time: 164303

active: True
id: GPGLL
latitude: 35.1958851667 degree
longitude: -106.708156167 degree
time: 164308



Note that :func:`pipecat.filter.keep` discards all records that don't meet the given criteria, which allows our downstream code to rely on the availability of specific fields.

Regardless of the logic you employ to identify fields of interest, Pipecat always makes it easy to convert units safely and explicitly:

In [11]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=120)

In [12]:
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
for record in pipe:
    if "speed" in record:
        print(record["speed"].to(pipecat.units.mph))

39.9320468464 mph
40.0586325857 mph
40.1276793526 mph
38.5626193033 mph


Let's explore other things we can do with our pipe.  To begin, you might want to add additional metadata to the records returned from a device.  For example, if you were collecting data from multiple devices you might want to "tag" records with a user-specific unique identifier:

In [13]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client, start=100, stop=115)

In [14]:
import pipecat.utility
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
pipe = pipecat.filter.keep(pipe, key="id", value="GPGLL")
pipe = pipecat.utility.add_field(pipe, "serial", "1237V")
for record in pipe:
    pipecat.record.dump(record)

active: True
id: GPGLL
latitude: 35.1949926667 degree
longitude: -106.7111135 degree
serial: 1237V
time: 164252

active: True
id: GPGLL
latitude: 35.1952843333 degree
longitude: -106.710192667 degree
serial: 1237V
time: 164257

active: True
id: GPGLL
latitude: 35.1956116667 degree
longitude: -106.709064 degree
serial: 1237V
time: 164303



Now let's consider calculating some simple statistics, such as our average speed on a trip.  When we iterate over the contents of a pipe using a `for` loop, we receive one record at-a-time until the pipe is empty.  We could keep track of a "running" average during iteration, and there are use-cases where that is the best way to solve the problem.  However, for moderately-sized data, Pipecat provides a more convenient approach: 

In [15]:
# nbconvert: hide
socket.socket().recvfrom.side_effect = pipecat.test.recvfrom_file(path=path, client=client)

In [16]:
import pipecat.store
pipe = pipecat.udp.receive(address=("0.0.0.0", 7777), maxsize=1024)
pipe = pipecat.device.gps.nmea(pipe)
pipe = pipecat.store.cache(pipe)
for record in pipe:
    pass
print(pipe.table["speed"])

[  0.     0.    10.59  17.96   4.39  24.14  30.65  33.59  33.28  32.85  34.08  34.78  35.28  34.66  34.46  34.1   34.64  34.41  33.88  33.75  34.7   34.81  34.87  33.51  33.71  35.38  32.09  28.94  18.     0.     1.19  21.23  31.92  33.55  34.91  34.78  33.75  32.71  31.67  31.14  31.45  31.94  31.16  32.27  35.46  35.34  34.06  33.82  34.91  34.72  34.83  34.95  33.38  33.08  27.39   5.21   0.     2.45  22.68  33.1   33.8  34.64  33.96  34.37  34.81  32.75  29.55  21.71  13.8   14.48  27.29  25.21  11.68  13.86   9.16   0.  ] knot


Here, :func:`pipecat.store.cache` creates an in-memory cache that stores every record it receives.   We have a do-nothing `for` loop that reads data from the charger to populate the cache.  Once that's complete, we can use the cache `table` attribute to retrieve data from the cache using the same keys and syntax we would use with a record.  Unlike a record, the cache returns every value for a given key at once (using a Numpy array), which makes it easy to compute the statistics we're interested in:

In [17]:
print("Average speed:", pipe.table["speed"].mean().to(pipecat.units.mph))

Average speed: 30.8533055116 mph


Consolidating fields using the cache is also perfect for generating plots with a library like Toyplot (http://toyplot.readthedocs.io):

In [18]:
import toyplot

canvas = toyplot.Canvas(width=600, height=400)
axes = canvas.cartesian(grid=(2, 1, 0), xlabel="Record #", ylabel="Speed (MPH)")
axes.plot(pipe.table["speed"].to(pipecat.units.mph))
axes = canvas.cartesian(grid=(2, 1, 1), xlabel="Record #", ylabel="Track")
axes.plot(pipe.table["track"]);

<path d="M 50.0 150.0 L 56.666666666666671 150.0 L 63.333333333333343 120.13536379018612 L 70.0 99.351381838691481 L 76.666666666666671 137.61985335589398 L 83.333333333333329 81.923293852227857 L 90.0 63.564579808234647 L 96.666666666666657 55.273547659334461 L 103.33333333333334 56.147772137619853 L 110.0 57.360406091370557 L 116.66666666666666 53.891708967851102 L 123.33333333333334 51.917653694303446 L 130.0 50.507614213197982 L 136.66666666666669 52.256063169768758 L 143.33333333333334 52.820078962210935 L 150.0 53.835307388606878 L 156.66666666666669 52.312464749012975 L 163.33333333333331 52.961082910321501 L 170.0 54.455724760293279 L 176.66666666666669 54.822335025380717 L 183.33333333333331 52.143260011280297 L 190.00000000000003 51.833051325437125 L 196.66666666666669 51.663846587704455 L 203.33333333333331 55.499153976311348 L 210.0 54.935138183869149 L 216.66666666666666 50.225606316976879 L 223.33333333333334 59.503666102650875 L 230.0 68.38691483361535 L 236.66666666666669 99.238578680203062 L 243.33333333333331 150.0 L 250.0 146.64410603496898 L 256.66666666666669 90.129723632261715 L 263.33333333333337 59.983079526226739 L 270.0 55.386350817822908 L 276.66666666666663 51.55104342921603 L 283.33333333333337 51.917653694303446 L 290.0 54.822335025380717 L 296.66666666666663 57.755217146080085 L 303.33333333333337 60.688099266779474 L 310.0 62.182741116751274 L 316.66666666666663 61.308516638465868 L 323.33333333333331 59.926677946982522 L 330.00000000000006 62.12633953750705 L 336.66666666666669 58.996051889452893 L 343.33333333333337 50.0 L 350.0 50.338409475465298 L 356.66666666666663 53.948110547095311 L 363.33333333333337 54.624929498025942 L 370.0 51.55104342921603 L 376.66666666666663 52.086858432036109 L 383.33333333333331 51.776649746192902 L 390.0 51.438240270727569 L 396.66666666666669 55.86576424139875 L 403.33333333333337 56.711787930062044 L 410.0 72.758037225042301 L 416.66666666666663 135.30738860688101 L 423.33333333333337 150.0 L 430.0 143.09080654258318 L 436.66666666666663 86.040609137055853 L 443.33333333333331 56.655386350817814 L 450.0 54.68133107727018 L 456.66666666666669 52.312464749012975 L 463.33333333333337 54.230118443316407 L 470.0 53.073886068809948 L 476.66666666666669 51.833051325437125 L 483.33333333333337 57.642413987591659 L 490.0 66.666666666666671 L 496.66666666666663 88.776085730400467 L 503.33333333333331 111.082910321489 L 510.0 109.16525662718557 L 516.66666666666674 73.040045121263404 L 523.33333333333326 78.905809362662154 L 530.0 117.06147772137621 L 536.66666666666674 110.91370558375635 L 543.33333333333326 124.16807670614779 L 550.0 150.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 25 50 75 Record # 0 10 20 30 40 Speed (MPH) <path d="M 50.0 350.0 L 56.666666666666671 349.25149700598803 L 63.333333333333343 342.2155688622754 L 70.0 348.20359281437123 L 76.666666666666671 343.8922155688623 L 83.333333333333329 323.41317365269458 L 90.0 323.17365269461072 L 96.666666666666657 323.23353293413174 L 103.33333333333334 323.05389221556885 L 110.0 322.63473053892216 L 116.66666666666666 324.1317365269461 L 123.33333333333334 327.69461077844306 L 130.0 330.56886227544913 L 136.66666666666669 329.82035928143716 L 143.33333333333334 329.79041916167665 L 150.0 329.10179640718565 L 156.66666666666669 329.10179640718565 L 163.33333333333331 329.10179640718565 L 170.0 329.25149700598803 L 176.66666666666669 329.19161676646706 L 183.33333333333331 328.44311377245509 L 190.00000000000003 328.95209580838326 L 196.66666666666669 329.16167664670661 L 203.33333333333331 329.16167664670661 L 210.0 329.04191616766462 L 216.66666666666666 327.90419161676652 L 223.33333333333334 326.19760479041918 L 230.0 324.64071856287421 L 236.66666666666669 323.59281437125748 L 243.33333333333331 323.47305389221555 L 250.0 323.47305389221555 L 256.66666666666669 296.85628742514973 L 263.33333333333337 296.22754491017963 L 270.0 292.21556886227546 L 276.66666666666663 284.0

Note that nothing prevents us from doing useful work in the `for` loop that populates the cache, and nothing prevents us from accessing the cache within the loop.  For example, we might want to display field values from individual records alongside a running average computed from the cache.  Or we might want to update our plot periodically as the loop progresses.

In [19]:
# nbconvert: stop

Moving on, you will likely want to store records to disk for later access.  Pipecat provides components to make this easy too.  First, you can add :func:`pipecat.store.pickle.write` to the end of a pipe, to write records to disk using Python's pickle format:

This is another example of the interchangeability of the Pipecat components: the pickle writer is a record consumer, and the pickle reader is a record generator.  In essence, we "broke" our previous pipe into two separate pipes that communicate via the filesystem.  While we won't go into detail here, a similar approach could be used to communicate between threads using a message queue or between processes over a socket.

There is one final issue that we've ignored so far: when to stop logging data.  The :func:`pipecat.utility.readline()` function will read data from the serial port as long as the port is open, blocking indefinitely if no data is arriving.  That means that for all the preceeding examples the `for` loop will never end unless the serial port is closed (i.e. the external device is turned-off or unplugged), or the code is interrupted using Control-C.  While that's fine for prototyping at the command line, we need to have a way to stop collecting data and shutdown cleanly if we're going to automate data logging processes.  Fortunately, Pipecat provides several easy-to-use functions to do just that:

Here, :func:`pipecat.limit.count` ends the loop when `count` records have been received.  This is often handy during development to limit the amount of data consumed from a device that produces output continuously.  However, this approach is no good for devices like our charger that will produce a finite, indeterminate number of records - if the device stops sending records before the `count` has been reached, the loop will still block.  Instead, we could use :func:`pipecat.limit.duration` to limit the total amount of time the loop is allowed to run instead:

This approach is an improvement because it puts an upper-bound on the amount of time the loop will run, whether the device has stopped sending records or not.  However, it's still error-prone, since we don't know in advance how long charging will take - if we set the duration too low, it may stop the loop before charging is complete.  If we set the duration too high, we will capture all the records we want, but we will likely waste time waiting for records that will never come.  Ideally, we would like to exit the loop as soon as the charger tells us it's finished.  Fortunately, the charger provides a field - `charger/mode` that can do just that:

:func:`pipecat.limit.until` terminates the loop as soon as it receives a record with the given key and value.  This approach finally gets us our desired behavior (loop ends as soon as the charger is finished), but it could use just a little more work to make it robust.  For example, what happens if the charger stops sending data before the mode changes?  We could combine :func:`pipecat.limit.until` with :func:`pipecat.limit.duration`, but that would still suffer from the terminate-too-soon / waste-too-much-time problem.  Fortunately, we know from testing that our charger sends records every two seconds (if at all), and Pipecat provides :func:`pipecat.limit.timeout`, which can terminate the loop if it doesn't receive a record within a specified time interval: 

While this example exits normally, you can see how easy it is to combine limit functions to control when the `for` loop terminates.